## 과제 1
ReLu activation function과 derivative function을 구현해보세요
- Hint : np.maximum 함수 사용하면 편리합니다
- 다른 방법 사용하셔도 무방합니다


In [3]:
import numpy as np
def relu(x):
    return np.maximum(0,x)
print(relu(2))

2


In [4]:
def d_relu(x):
    if x < 0:
        return 0
    else:
        return 1

print(d_relu(2))

1


## 과제 2
Deep Learning Basic 코드 파일의 MLP implementation with Numpy library using MNIST dataset 코드 참고해서
Three layer MLP 일 때의 backward_pass 함수를 완성해주세요.   
- Hint : 코드 파일의 예시는 Two layer MLP


In [5]:
def backward_pass(x, y_true, params):

    dS3 = params["A3"] - y_true

    grads = {}

    grads["dW3"] =  np.dot(dS3, params["A2"].T)/x.shape[1]
    grads["db3"] =  (1/x.shape[1])*np.sum(dS3, axis=1, keepdims=True)/x.shape[1]

    grads["dW2"] =  np.dot(dS2, params["A1"].T)/x.shape[1]
    grads["db2"] =  (1/x.shape[1])*np.sum(dS2, axis=1, keepdims=True)/x.shape[1]

    dA1 = np.dot(params["W2"].T, dS2)
    dS1 = dA1 * d_sigmoid(params["S1"])

    grads["dW1"] = np.dot(dS1, x.T)/x.shape[1]
    grads["db1"] = np.sum(dS1, axis=1, keepdims=True)/x.shape[1]

    return grads

## 과제 3
Deep Learning Basic 코드 파일의 MLP implementation with Pytorch library using MNIST dataset 코드 참고해서
Three layer MLP를 구한후, 학습을 돌려 보세요

hyperparameter는 다음과 같이 설정

- epochs : 100
- hiddensize : 128, 64 (two layer)
- learning_rate : 0.5

In [6]:
from torchvision import transforms, datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [7]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [8]:
trainset = datasets.MNIST(
    root      = './.data/', 
    train     = True,
    download  = True,
    transform = transform
)
testset = datasets.MNIST(
    root      = './.data/', 
    train     = False,
    download  = True,
    transform = transform
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./.data/MNIST\raw\train-images-idx3-ubyte.gz to ./.data/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./.data/MNIST\raw\train-labels-idx1-ubyte.gz to ./.data/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./.data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./.data/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./.data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./.data/MNIST\raw



In [9]:
BATCH_SIZE = 512

train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader =  DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(784,128)
        self.layer2 = nn.Linear(128,64)
        self.layer3 = nn.Linear(64,10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.relu(out)
        out = self.layer3(out)

        return out

In [11]:
model = Net()
model

Net(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [12]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0169, -0.0306, -0.0293,  ...,  0.0024,  0.0168, -0.0325],
         [ 0.0265,  0.0076, -0.0232,  ...,  0.0342, -0.0088,  0.0207],
         [ 0.0172,  0.0176, -0.0089,  ..., -0.0352, -0.0152,  0.0040],
         ...,
         [-0.0060, -0.0324,  0.0348,  ...,  0.0028, -0.0220,  0.0205],
         [-0.0238, -0.0005, -0.0208,  ..., -0.0102, -0.0267, -0.0141],
         [ 0.0273,  0.0270, -0.0103,  ..., -0.0306, -0.0317,  0.0123]],
        requires_grad=True),
 Parameter containing:
 tensor([ 2.3096e-02, -3.1422e-02, -6.8047e-03,  1.3254e-02, -2.6889e-03,
         -3.4172e-02,  2.7767e-02,  2.6929e-02, -2.4585e-02, -2.7267e-02,
          2.1142e-02, -9.7406e-03, -4.7191e-03, -3.4670e-02,  1.8651e-02,
          2.0922e-02, -8.9046e-03,  1.3520e-02, -1.9148e-02,  2.5238e-02,
         -1.2193e-02, -2.8478e-02, -5.2660e-03,  2.8169e-02, -5.1021e-03,
         -1.2769e-02, -3.4826e-02,  5.2888e-05, -1.4744e-02,  3.4501e-02,
          3.1735e-02, -9.5470e-03,  1.39

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.5)

In [14]:
def train(model, train_loader, optimizer):
    model.train()
    batch_losses = []
    for data, target in train_loader:
        
        optimizer.zero_grad()

        output = model(data)
        loss = criterion(output, target)
        batch_losses.append(loss)

        loss.backward()
        
        optimizer.step()
        
    avg_loss = sum(batch_losses) / len(batch_losses)
    
    return avg_loss

In [16]:
def evaluate(model, test_loader):
    model.eval()
    batch_losses = []
    correct = 0 
    with torch.no_grad(): 
        for data, target in test_loader:
            output = model(data)
            loss = criterion(output, target)
            batch_losses.append(loss)     
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    avg_loss =  sum(batch_losses) / len(batch_losses)
    accuracy = 100. * correct / len(test_loader.dataset)

    return avg_loss, accuracy

In [17]:
EPOCHS = 100

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
          epoch, train_loss, test_loss, test_accuracy))

[1] Train Loss: 0.8271	Test Loss: 0.3808	Accuracy: 87.68%
[2] Train Loss: 0.2517	Test Loss: 0.1967	Accuracy: 93.95%
[3] Train Loss: 0.1715	Test Loss: 0.1615	Accuracy: 94.79%
[4] Train Loss: 0.1350	Test Loss: 0.1588	Accuracy: 95.30%
[5] Train Loss: 0.1107	Test Loss: 0.1328	Accuracy: 95.64%
[6] Train Loss: 0.0938	Test Loss: 0.1250	Accuracy: 95.87%
[7] Train Loss: 0.0804	Test Loss: 0.2637	Accuracy: 91.95%
[8] Train Loss: 0.3573	Test Loss: 0.1660	Accuracy: 94.61%
[9] Train Loss: 0.0994	Test Loss: 0.1119	Accuracy: 96.40%
[10] Train Loss: 0.0794	Test Loss: 0.1208	Accuracy: 96.46%
[11] Train Loss: 0.0688	Test Loss: 0.2296	Accuracy: 93.30%
[12] Train Loss: 0.0633	Test Loss: 0.0807	Accuracy: 97.51%
[13] Train Loss: 0.0531	Test Loss: 0.1673	Accuracy: 95.00%
[14] Train Loss: 0.5788	Test Loss: 0.1832	Accuracy: 94.39%
[15] Train Loss: 0.1096	Test Loss: 0.1640	Accuracy: 94.82%
[16] Train Loss: 0.0864	Test Loss: 0.1143	Accuracy: 96.55%
[17] Train Loss: 0.0751	Test Loss: 0.1590	Accuracy: 95.22%
[18] T

## 과제 4
과제 3 부분의 성능을 지금까지 배운 지식을 바탕으로 향상시켜보세요

- Hint : Activation function, hyperparameter setting

In [25]:
BATCH_SIZE = 300

train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
test_loader =  DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(784,128)
        self.layer2 = nn.Linear(128,64)
        self.layer3 = nn.Linear(64,10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        out = self.relu(out)
        out = self.layer3(out)

        return out

In [27]:
model = Net()
model

Net(
  (layer1): Linear(in_features=784, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [28]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0017, -0.0350, -0.0138,  ...,  0.0004, -0.0261, -0.0273],
         [-0.0267, -0.0274,  0.0013,  ..., -0.0279,  0.0163, -0.0325],
         [-0.0050,  0.0159,  0.0221,  ...,  0.0110, -0.0108,  0.0058],
         ...,
         [ 0.0312,  0.0227, -0.0290,  ...,  0.0111, -0.0259, -0.0119],
         [ 0.0171,  0.0321,  0.0337,  ..., -0.0256,  0.0100, -0.0199],
         [-0.0297, -0.0092,  0.0235,  ...,  0.0024, -0.0073,  0.0195]],
        requires_grad=True),
 Parameter containing:
 tensor([ 1.6172e-02, -1.7031e-02,  1.0837e-02,  3.0965e-02, -5.4007e-03,
          1.8564e-02,  3.5060e-02, -2.1299e-02,  3.1358e-02,  2.3605e-02,
          1.9931e-02, -1.3445e-02, -3.7950e-03, -2.7798e-02, -3.5247e-02,
          8.0492e-03,  3.0886e-02, -2.0439e-02,  1.1902e-02,  1.4418e-02,
         -3.4872e-02,  2.9560e-02,  2.5756e-02, -2.5807e-02, -2.5183e-02,
         -1.6014e-02, -1.0088e-02,  7.4029e-03, -1.2149e-02, -2.6508e-02,
          1.3365e-02, -3.0130e-03,  2.72

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [30]:
def train(model, train_loader, optimizer):
    model.train()
    batch_losses = []
    for data, target in train_loader:
        
        optimizer.zero_grad()

        output = model(data)
        loss = criterion(output, target)
        batch_losses.append(loss)

        loss.backward()
        
        optimizer.step()
        
    avg_loss = sum(batch_losses) / len(batch_losses)
    
    return avg_loss

In [31]:
def evaluate(model, test_loader):
    model.eval()
    batch_losses = []
    correct = 0 
    with torch.no_grad(): 
        for data, target in test_loader:
            output = model(data)
            loss = criterion(output, target)
            batch_losses.append(loss)     
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    avg_loss =  sum(batch_losses) / len(batch_losses)
    accuracy = 100. * correct / len(test_loader.dataset)

    return avg_loss, accuracy

In [32]:
EPOCHS = 70

for epoch in range(1, EPOCHS + 1):
    train_loss = train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Train Loss: {:.4f}\tTest Loss: {:.4f}\tAccuracy: {:.2f}%'.format(
          epoch, train_loss, test_loss, test_accuracy))

[1] Train Loss: 1.1510	Test Loss: 0.4518	Accuracy: 87.07%
[2] Train Loss: 0.3752	Test Loss: 0.3264	Accuracy: 90.37%
[3] Train Loss: 0.3028	Test Loss: 0.2731	Accuracy: 92.27%
[4] Train Loss: 0.2613	Test Loss: 0.2373	Accuracy: 93.31%
[5] Train Loss: 0.2285	Test Loss: 0.2057	Accuracy: 94.17%
[6] Train Loss: 0.2026	Test Loss: 0.1852	Accuracy: 94.66%
[7] Train Loss: 0.1820	Test Loss: 0.1701	Accuracy: 94.98%
[8] Train Loss: 0.1632	Test Loss: 0.1552	Accuracy: 95.52%
[9] Train Loss: 0.1484	Test Loss: 0.1415	Accuracy: 95.83%
[10] Train Loss: 0.1352	Test Loss: 0.1377	Accuracy: 95.90%
[11] Train Loss: 0.1243	Test Loss: 0.1252	Accuracy: 96.33%
[12] Train Loss: 0.1150	Test Loss: 0.1159	Accuracy: 96.61%
[13] Train Loss: 0.1063	Test Loss: 0.1135	Accuracy: 96.59%
[14] Train Loss: 0.0984	Test Loss: 0.1070	Accuracy: 96.91%
[15] Train Loss: 0.0925	Test Loss: 0.1024	Accuracy: 96.93%
[16] Train Loss: 0.0860	Test Loss: 0.0996	Accuracy: 97.02%
[17] Train Loss: 0.0808	Test Loss: 0.0953	Accuracy: 97.14%
[18] T

**무엇을 보완하였고, 왜 보완되었는지에 대한 자유 서술 (아래에)**

In [ ]:
과적합의 문제를 해결하고자 Batch Size의 크기를 줄여보았습니다. #512->300 
마찬가지로 과적합 문제를 해결하고자 epoch를 줄여보았습니다. #100->70
오버슈팅의 문제로 인해서 생기는 train loss 이슈를 해결하기 위해 learning rate 를 줄여보았습니다. #0.5->0.1

결과적으로 전체적인 accuracy는 약간 높아진 것으로 보입니다.